# APIM ❤️ MCP

## MCP Client Authorization lab

![flow](../../images/mcp-client-authorization.gif)

Playground to experiment the [Model Context Protocol](https://modelcontextprotocol.io/) with the [client authorization flow](https://modelcontextprotocol.io/specification/2025-03-26/basic/authorization#2-10-third-party-authorization-flow). In this flow, Azure API Management act both as an OAuth client connecting to the [Microsoft Entra ID](https://learn.microsoft.com/en-us/entra/architecture/auth-oauth2) authorization server and as an OAuth authorization server for the MCP client ([MCP inspector](https://modelcontextprotocol.io/docs/tools/inspector) in this lab).


This [sequence diagram](./diagrams/diagrams.md) explains the flow. 

⚠️ Due to the evolving nature of the [MCP Authorization proposal](https://modelcontextprotocol.io/specification/2025-03-26/basic/authorization), direct use of this implementation in production environments is not yet recommended.

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json, base64
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "uksouth"

apim_sku = 'Basicv2'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

app_registration_name = "mcp-app-registration-01"

build = 0
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "src/weather/mcp-server"

# In this lab we will generate AES keys for encryption and decryption of tokens.
# This is an experimental feature and should NOT be used in production!
encryption_iv = base64.b64encode(os.urandom(16)).decode('utf-8')
encryption_key = base64.b64encode(os.urandom(16)).decode('utf-8')
oauth_scopes = 'openid https://graph.microsoft.com/.default'

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create the App Registration in Microsoft Entra ID

The following command creates a client application registration

In [ ]:
# Let's try a more direct approach with Azure CLI
import subprocess
import json

# Try to create app registration with explicit JSON output
unique_app_name = f"{app_registration_name}-{str(int(time.time()))[-4:]}"
print(f"Creating app registration: {unique_app_name}")

try:
    # First create the app registration
    create_cmd = f'az ad app create --display-name "{unique_app_name}" --is-fallback-public-client true --output json'
    print(f"Running: {create_cmd}")
    
    result = subprocess.run(create_cmd, shell=True, capture_output=True, text=True)
    print(f"Return code: {result.returncode}")
    print(f"Stdout: {result.stdout}")
    print(f"Stderr: {result.stderr}")
    
    if result.returncode == 0 and result.stdout:
        app_data = json.loads(result.stdout)
        entraid_client_id = app_data.get('appId')
        print(f"✅ Created app with ID: {entraid_client_id}")
        
        # Now create a client secret
        if entraid_client_id:
            secret_cmd = f'az ad app credential reset --id "{entraid_client_id}" --only-show-errors --output json'
            print(f"Creating client secret...")
            
            secret_result = subprocess.run(secret_cmd, shell=True, capture_output=True, text=True)
            if secret_result.returncode == 0 and secret_result.stdout:
                secret_data = json.loads(secret_result.stdout)
                entraid_client_secret = secret_data.get('password')
                print(f"✅ Created client secret")
            else:
                print(f"❌ Failed to create client secret: {secret_result.stderr}")
                entraid_client_secret = None
        else:
            entraid_client_secret = None
    else:
        print(f"❌ Failed to create app: {result.stderr}")
        entraid_client_id = None
        entraid_client_secret = None
        
except Exception as e:
    print(f"❌ Exception: {e}")
    entraid_client_id = None
    entraid_client_secret = None

print(f"👉🏻 Final Client Id: {entraid_client_id}")
print(f"👉🏻 Final Client Secret: {'***' if entraid_client_secret else 'None'}")

# Update the app registration name for consistency
app_registration_name = unique_app_name

<a id='3'></a>
### 3️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIAPIVersion": { "value": openai_api_version },
        "entraIDClientId": { "value": entraid_client_id },
        "entraIDClientSecret": { "value": entraid_client_secret },
        "oauthScopes": { "value": oauth_scopes },
        "encryptionIV": { "value": encryption_iv },
        "encryptionKey": { "value": encryption_key },
        "mcpClientId": { "value": entraid_client_id }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='4'></a>
### 4️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Try a direct approach to get deployment outputs
import subprocess
import json

try:
    # Get deployment outputs directly
    cmd = f'az deployment group show --name {deployment_name} -g {resource_group_name} --output json'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        deployment_data = json.loads(result.stdout)
        outputs = deployment_data.get('properties', {}).get('outputs', {})
        
        print("Available deployment outputs:")
        for key, value in outputs.items():
            print(f"  {key}: {value.get('value', 'No value')}")
        
        # Extract the required outputs
        container_registry_name = outputs.get('containerRegistryName', {}).get('value')
        weather_containerapp_resource_name = outputs.get('weatherMCPServerContainerAppResourceName', {}).get('value')
        apim_resource_gateway_url = outputs.get('apimResourceGatewayURL', {}).get('value')
        apim_resource_name = outputs.get('apimResourceName', {}).get('value')
        apim_subscription_key = outputs.get('apimSubscriptionKey', {}).get('value')
        apim_service_id = outputs.get('apimServiceId', {}).get('value')
        app_insights_name = outputs.get('applicationInsightsName', {}).get('value')
        
        print(f"\n✅ Successfully extracted deployment outputs:")
        print(f"  container_registry_name: {container_registry_name}")
        print(f"  weather_containerapp_resource_name: {weather_containerapp_resource_name}")
        print(f"  apim_resource_gateway_url: {apim_resource_gateway_url}")
        print(f"  apim_resource_name: {apim_resource_name}")
        print(f"  app_insights_name: {app_insights_name}")
        
    else:
        print(f"❌ Failed to get deployment outputs: {result.stderr}")
        
except Exception as e:
    print(f"❌ Exception occurred: {e}")

<a id='5'></a>
### 5️⃣ Build and deploy the MCP Servers



In [ ]:
build = build + 1 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
          "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
          "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")


<a id='6'></a>
### 6️⃣ Update the App Registration with the Redirect URI from APIM



In [ ]:
utils.run(f"az ad app update --id {entraid_client_id} --web-redirect-uris \"{apim_resource_gateway_url}/oauth-callback\"",
          "Added redirect URL to app registration",
          "Failed to add redirect URL to app registration")

<a id='unauthorizedtest'></a>
### 🧪 Test the authorization **WITHOUT** a valid token

In [ ]:
# Unauthenticated call should fail with 401 Unauthorized
import requests

mcp_server_url = f"{apim_resource_gateway_url}/weather/sse"
utils.print_info("Calling sse endpoint WITHOUT authorization...")
utils.print_message(f"MCP Server Url : {mcp_server_url}")
response = requests.get(mcp_server_url, headers={"Content-Type": "application/json"})
if response.status_code == 401:
    utils.print_ok("Received 401 Unauthorized as expected")
elif response.status_code == 200:
    utils.print_error("Call succeeded. Double check that validate-jwt policy has been deployed to sse endpoint")
else:
    utils.print_error(f"Unexpected status code: {response.status_code}")


<a id='inspector'></a>
### 🧪 Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) to test the Authorization flow

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Access the provided URL in a browser, replacing the IP address with 'localhost'.
3. Set the transport type as SSE
4. Provide the MCP server url and click connect
5. You should see a sign-in screen or a prompt to consent to the use of the currently signed-in account
6. Select the "Tools" tab to view and execute the available tools

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.